In [3]:
from dataset import *

In [4]:
def pil_to_numpy(im):
    """
    Converts a PIL Image object to a NumPy array.
    Source : Fast import of Pillow images to NumPy / OpenCV arrays Written by Alex Karpinsky

    Args:
        im (PIL.Image.Image): The input PIL Image object.

    Returns:
        numpy.ndarray: The NumPy array representing the image.
    """
    im.load()

    # Unpack data
    e = Image._getencoder(im.mode, "raw", im.mode)
    e.setimage(im.im)

    # NumPy buffer for the result
    shape, typestr = Image._conv_type_shape(im)
    data = np.empty(shape, dtype=np.dtype(typestr))
    mem = data.data.cast("B", (data.data.nbytes,))

    bufsize, s, offset = 65536, 0, 0
    while not s:
        l, s, d = e.encode(bufsize)

        mem[offset : offset + len(d)] = d
        offset += len(d)
    if s < 0:
        raise RuntimeError("encoder error %d in tobytes" % s)
    return data

In [13]:
from PIL import Image

image_name = (
    "/home/ikergarcia/data/gtaai_datasets/test/car_city_day_clear/K0%3_3_3_5_5.jpeg"
)
# Read image into numpy array
image_splitter = SplitImages()
rm = RemoveMinimap(hide_map_prob=1)
image = Image.open(image_name)
image = pil_to_numpy(image)
# image = rm(image)
img = Image.fromarray(image, "RGB")
img.show()
image.shape

(270, 2400, 3)

In [11]:
image = image_splitter(image)

img = Image.fromarray(image[0], "RGB")
img.show()

In [16]:
tm = TubeMaskingGenerator((5, 270, 480), 0.9)

In [17]:
mask = tm()

In [18]:
mask.size()

torch.Size([648000])

In [13]:
from transformers import VideoMAEImageProcessor

In [14]:
image_processor = VideoMAEImageProcessor(
    do_resize=False,
    do_center_crop=False,
    do_rescale=True,
    do_normalize=True,
    image_mean=[0.33363932, 0.32581538, 0.31566033],
    image_std=[0.1914285, 0.18449214, 0.1853477],
)

In [25]:
model_inputs["pixel_values"].size()

torch.Size([1, 5, 3, 270, 480])

In [21]:
len(list(image))

5

In [1]:
from transformers import AutoImageProcessor, VideoMAEForPreTraining
import numpy as np
import torch

num_frames = 16
video = list(np.random.randint(0, 256, (num_frames, 3, 224, 224)))

image_processor = AutoImageProcessor.from_pretrained("MCG-NJU/videomae-base")
model = VideoMAEForPreTraining.from_pretrained(
    "MCG-NJU/videomae-base", torch_dtype=torch.bfloat16
).to("cuda")

pixel_values = image_processor(video, return_tensors="pt").pixel_values

num_patches_per_frame = (model.config.image_size // model.config.patch_size) ** 2
seq_length = (num_frames // model.config.tubelet_size) * num_patches_per_frame
bool_masked_pos = torch.randint(0, 2, (1, seq_length)).bool()

outputs = model(
    pixel_values.to(device=model.device, dtype=model.dtype),
    bool_masked_pos=bool_masked_pos,
)
loss = outputs.loss

loss.backward()

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.
/home/ikergarcia/miniconda3/envs/transformers/lib/python3.10/site-packages/transformers/feature_extraction_utils.py:141: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /opt/conda/conda-bld/pytorch_1694072516410/work/torch/csrc/utils/tensor_new.cpp:261.)
  return torch.tensor(value)


logits: torch.bfloat16
labels: torch.float32
loss: torch.float32


RuntimeError: Found dtype Float but expected BFloat16

In [3]:
from modeling_videomae import VideoMAEForPreTraining
import torch

model = VideoMAEForPreTraining.from_pretrained(
    "models/VideoMAE-base", torch_dtype=torch.bfloat16
)

Some weights of VideoMAEForPreTraining were not initialized from the model checkpoint at models/VideoMAE-base and are newly initialized: ['decoder.decoder_layers.5.layernorm_before.weight', 'decoder.decoder_layers.0.attention.output.dense.bias', 'decoder.decoder_layers.5.attention.attention.v_bias', 'decoder.decoder_layers.0.attention.attention.key.weight', 'decoder.decoder_layers.5.layernorm_before.bias', 'decoder.decoder_layers.0.layernorm_before.weight', 'decoder.decoder_layers.0.output.dense.weight', 'decoder.decoder_layers.1.layernorm_after.bias', 'decoder.decoder_layers.3.attention.attention.v_bias', 'decoder.decoder_layers.2.output.dense.weight', 'decoder.decoder_layers.4.attention.output.dense.weight', 'decoder.decoder_layers.3.attention.attention.key.weight', 'decoder.decoder_layers.3.attention.output.dense.weight', 'decoder.decoder_layers.1.attention.output.dense.bias', 'decoder.decoder_layers.3.layernorm_after.weight', 'decoder.decoder_layers.4.attention.attention.key.weight

In [4]:
model.config

VideoMAEConfig {
  "_name_or_path": "models/VideoMAE-base",
  "architectures": [
    "VideoMAEModel"
  ],
  "attention_probs_dropout_prob": 0.0,
  "decoder_hidden_size": 384,
  "decoder_intermediate_size": 1536,
  "decoder_num_attention_heads": 6,
  "decoder_num_hidden_layers": 6,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 768,
  "image_size": [
    270,
    480
  ],
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "model_type": "videomae",
  "norm_pix_loss": true,
  "num_attention_heads": 12,
  "num_channels": 3,
  "num_frames": 5,
  "num_hidden_layers": 12,
  "patch_size": 15,
  "qkv_bias": true,
  "torch_dtype": "float32",
  "transformers_version": "4.35.0",
  "tubelet_size": 1,
  "use_mean_pooling": true
}

In [1]:
from dataset import *

In [1]:
from transformers import VideoMAEImageProcessor as original_image_processor
from image_processing_videomae import VideoMAEImageProcessor as new_image_processor
import torch

In [2]:
IMAGE_MEAN: list[float] = [0.33363932, 0.32581538, 0.31566033]
IMAGE_STD: list[float] = [0.1914285, 0.18449214, 0.1853477]

image_processor_og = original_image_processor(
    do_resize=False,
    do_center_crop=False,
    do_rescale=True,
    do_normalize=True,
    image_mean=IMAGE_MEAN,
    image_std=IMAGE_STD,
)

image_processor_new = new_image_processor(
    do_resize=False,
    do_center_crop=False,
    do_rescale=True,
    do_normalize=True,
    image_mean=IMAGE_MEAN,
    image_std=IMAGE_STD,
)

In [3]:
import numpy as np

In [5]:
image_sequences = np.asarray(
                    [
                        np.random.rand(200, 200, 3)*255,
                    ]*128,
                    dtype=np.uint8,
                )

image_sequences = list(image_sequences)
print(len(image_sequences))
print(image_sequences[0].shape)
print(image_sequences[0][0][0][:10])




128
(200, 200, 3)
[119 129 132]


In [6]:
%%timeit
model_inputs_og = image_processor_og(
                        images=image_sequences,
                        input_data_format="channels_last",
                        return_tensors="pt",
                    )

/home/iker/miniconda3/envs/transformers/lib/python3.11/site-packages/transformers/feature_extraction_utils.py:141: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:261.)
  return torch.tensor(value)


1.11 s ± 123 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [7]:
%%timeit
model_inputs_new = image_processor_new(
                        images=image_sequences,
                        input_data_format="channels_last",
                        return_tensors="pt",
                    )

154 ms ± 2.26 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [8]:
model_inputs_og = image_processor_og(
                        images=image_sequences,
                        input_data_format="channels_last",
                        return_tensors="pt",
                    )
model_inputs_og["pixel_values"].size()

torch.Size([1, 128, 3, 200, 200])

In [9]:
model_inputs_new = image_processor_new(
                        images=image_sequences,
                        input_data_format="channels_last",
                        return_tensors="pt",
                    )

model_inputs_new["pixel_values"].size()

torch.Size([1, 128, 3, 200, 200])

In [12]:
torch.all(model_inputs_new["pixel_values"]==model_inputs_og["pixel_values"])

tensor(True)

torch.Size([1, 5, 3, 270, 480])

In [3]:
from dataset import Tedd1104Dataset

In [4]:
dataset = Tedd1104Dataset(dataset_dir="C:\\Users\\igarc\\Downloads\\TEDD1104_Dev_Test\\test\\car_city_day_clear",hide_map_prob=0.0,mask_ratio=0.0,patch_size=30, tubelet_size = 5)

In [7]:
dataset[0]["pixel_values"].size()

torch.Size([5, 3, 270, 480])

In [8]:
import torch

In [10]:
 torch.zeros([1,9], dtype=torch.long)

tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0]])